In [1]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aayus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# Preprocessing

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() 
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aayus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

347 documents
36 classes ['admission', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'document', 'event', 'extchod', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'sem', 'swear', 'syllabus', 'task', 'uniform', 'vacation']
240 unique lemmatized words ["'s", 'a', 'ab1', 'about', 'active', 'address', 'admision', 'admission', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'as', 'asshole', 'at', 'attend', 'available', 'average', 'be', 'between', 'big', 'bitch', 'book', 'branch', 'bring', 'building', 'by', 'bye', 'cafetaria', 'call', 'called', 'campus', 'can', 'canteen', 'capacity', 'case', 'casuals', 'chatting', 'chennai', 'code', 'college', 'come', 'committe', 'committee', 'company', 'computer', 'conducted', 'contact', 'course', 'create', 'created', 'creator', 'cse',

In [8]:
import random
import numpy as np
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [9]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [10]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
70/70 [==============================] - 0s 1ms/step - loss: 3.5765 - accuracy: 0.0548
Epoch 2/200
70/70 [==============================] - 0s 1ms/step - loss: 3.4353 - accuracy: 0.0778
Epoch 3/200
70/70 [==============================] - 0s 1ms/step - loss: 3.3383 - accuracy: 0.1182
Epoch 4/200
70/70 [==============================] - 0s 1ms/step - loss: 3.1221 - accuracy: 0.1844
Epoch 5/200
70/70 [==============================] - 0s 1ms/step - loss: 2.9015 - accuracy: 0.2161
Epoch 6/200
70/70 [==============================] - 0s 1ms/step - loss: 2.6062 - accuracy: 0.3343
Epoch 7/200
70/70 [==============================] - 0s 1ms/step - loss: 2.3576 - accuracy: 0.3660
Epoch 8/200
70/70 [==============================] - 0s 1ms/step - loss: 2.1936 - accuracy: 0.4179
Epoch 9/200
70/70 [==============================] - 0s 1ms/step - loss: 1.8623 - accuracy: 0.5072
Epoch 10/200
70/70 [==============================] - 0s 1ms/step - loss: 1.6982 - accuracy: 0.5504
Epoch 11/

In [13]:
from keras.models import load_model
model = load_model('chatbot_model_v2.h5')

In [14]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words_v2.pkl','rb'))
classes = pickle.load(open('classes_v2.pkl','rb'))

In [15]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):    
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [16]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [17]:
res = chatbot_response('who is computer dept hod')
print(res)

Dr. Ganesan R is computer Dean


In [18]:
res=chatbot_response('what is your name')
print(res)

You can call me VITBOT.


In [19]:
res=chatbot_response('who made you')
print(res)

I was developed by Aayush, Monark and raghav


In [20]:
res=chatbot_response('what is FAT exam schedule?')
print(res)

Here is the Academic Calendar  <a target="_blank" href="https://vit.ac.in/sites/default/files/Fall_Semester_2021-22_Academic_Calendar.pdf">website</a>


In [21]:
res=chatbot_response('what are the events in VIT?')
print(res)

There are a lot of events of which VIBRANCE is the biggest and well celebrated one!!


In [22]:
res=chatbot_response('what courses does VIT offer?')
print(res)

VIT Chennai offers CSE, ECE, ECM, MECH, CIVIL, EEE and many more Branches


In [23]:
res=chatbot_response('How are placements in VIT?')
print(res)

To know about placement visit <a target="_blank" href="https://collegedunia.com/university/56865-vellore-institute-of-technology-vit-chennai/placement">here</a>


In [24]:
res=chatbot_response('Thank you')
print(res)

I am glad I helped you
